In [1]:
%cd ../../..

/workspaces/Code


In [2]:
from __future__ import annotations

import math
import gc
from typing import Iterator
import multiprocessing
from PIL import Image
import os

from image_provider import ImageProvider
from test_runner import TestRun
from map_provider import MapProvider, ImageProjection
from DroneProvider import SimulatedDroneProvider
from vector import Vector2D

In [3]:
crop_size = 448
accuracy_threshold = 25
particle_count = 100
particle_randomize_percent = 0

drone_last_known_noise = 50
initial_location_offset = 25

repeat_count = 10
drone_steps = 25
graph_size = 1200

In [4]:
def get_distance_at_45d(distance: int) -> int:
    coord = math.sqrt(math.pow(distance, 2) / 2)
    return math.floor(coord)

def build_drones(crop_size: int) -> Iterator[TestRun]:
    #6024x6024
    city_image = ImageProvider("City/NewTraining/ExperimentZone/City_2017.jpg")
    drone_image = ImageProvider("City/NewTraining/ExperimentZone/City_2016.jpg")

    projection = ImageProjection(Vector2D(0, 524), Vector2D(5000, 5000))
    city_map = MapProvider(
        image_provider=city_image,
        crop_size=crop_size,
        projection=projection)
    
    drone_map = MapProvider(
        image_provider=drone_image,
        crop_size=crop_size,
        projection=projection)
    
    run_name = "Forest"
    for move_by in [60, 100]:
        yield TestRun(run_name, SimulatedDroneProvider(drone_map, Vector2D(1000, 2500), Vector2D(move_by, 0), drone_steps), city_map)

    projection = ImageProjection(Vector2D(524, 0), Vector2D(5000, 5000))
    city_map = MapProvider(
        image_provider=city_image,
        crop_size=crop_size,
        projection=projection)
    
    drone_map = MapProvider(
        image_provider=drone_image,
        crop_size=crop_size,
        projection=projection)
    
    run_name = "Residential"
    for move_by in [60, 100]:
        move_by_side = get_distance_at_45d(move_by)
        yield TestRun(run_name, SimulatedDroneProvider(drone_map, Vector2D(2500, 1000), Vector2D(move_by_side, move_by_side), drone_steps), city_map)

    projection = ImageProjection(Vector2D(0, 1000), Vector2D(5000, 5000))
    city_map = MapProvider(
        image_provider=city_image,
        crop_size=crop_size,
        projection=projection)
    
    drone_map = MapProvider(
        image_provider=drone_image,
        crop_size=crop_size,
        projection=projection)
    
    run_name = "Apartments"
    for move_by in [60, 100]:
        yield TestRun(run_name, SimulatedDroneProvider(drone_map, Vector2D(2500, 4000), Vector2D(0, -move_by), drone_steps), city_map)

In [5]:
for drone_route in build_drones(crop_size):
    route_name = drone_route.name
    drone = drone_route.drone
    city_map = drone_route.city_map

    image_folder = f"Data/ParticleFilter/Images/{crop_size}/{route_name}/{drone}"
    os.makedirs(image_folder, exist_ok=True)

    while drone.has_step():
        drone_position = drone.get_position()

        drone_image = Image.fromarray(drone.grab_image())
        city_image = Image.fromarray(city_map.get_cropped_image(drone_position.x, drone_position.y))

        drone_image.save(f"{image_folder}/{drone.get_current_step():02d}_drone.jpg")
        city_image.save(f"{image_folder}/{drone.get_current_step():02d}_city.jpg")

        drone.move_step()



Loading image: City/NewTraining/ExperimentZone/City_2017.jpg
Loading image: City/NewTraining/ExperimentZone/City_2016.jpg
